# Работа с базой данных MongoDB

_Петр Ромов_

## Формат JSON

[Описание на Википедии](https://ru.wikipedia.org/wiki/JSON)

Позволяет описывать данные, которые не имеют строгой табличной структуры. Объекты (документы) записанные в формате JSON могут содержать:
 - произвольный набор полей
 - поля могут иметь один из типов:
   - строка
   - числовое значение
   - массив
   - вложенный объект
 - поля могут содержать "пустые" значения — `null`

**Пример**: запись в телефонной книге
```json
{
   "firstName": "Иван",
   "lastName": "Иванов",
   "address": {
       "streetAddress": "Московское ш., 101, кв.101",
       "city": "Ленинград",
       "postalCode": 101101
   },
   "phoneNumbers": [
       "812 123-1234",
       "916 123-4567"
   ]
}
```

## MongoDB

MongoDB — СУБД, которая позволяет работать с **базами данных** (database). База данных состоит из **коллекций** (collection). Коллекции содержат **документы** (documents). Документы представлены в формате JSON. 

Никаких ограничений на схему базы данных, формат коллекций не накладывается (в отличие от SQL-баз данных, в которых прежде чем записать данные, необходимо сначала задать схему таблиц).

Каждый документ содержит специальное поле `"_id"` (идентификатор) — это основной ключ, по которому происходит поиск документов в базе данных. Идентификатор уникален для каждого документа. Если вы не указываете поле `"_id"` при создании документа в коллекции, MongoDB создает его за вас. Если вы попытаетесь записать документ, у которого дублируется `"_id"` с другим объектом — получите ошибку от базы данных.

![](http://csharpcorner.mindcrackerinc.netdna-cdn.com/UploadFile/370e35/getting-started-with-mongodb-and-Asp-Net-mvc4-day-2/Images/MongoDB%20Collection.jpg)

##### Для работы с MongoDB в Python есть специальный модуль — импортируем его

In [1]:
import pymongo

##### Подключаемся к удаленной базе данных, в которой лежат данные

In [2]:
client = pymongo.MongoClient('goto.reproducible.work')

##### Выбираем нужную базу данных

В нашем случае — база данных, которая называется `vk`.

In [3]:
db = client['vk']

##### Список коллекций, которые содержит база данных

In [4]:
db.collection_names()

[u'friends', u'users', u'walls']

##### Можно посмотреть число документов в коллекции

In [5]:
db['users'].count()

730727

In [6]:
for collection_name in db.collection_names():
    print collection_name, db[collection_name].count()

friends 730727
users 730727
walls 730727


##### Возьмем произвольную запись из коллекции `users` и разберем ее содержимое

In [7]:
user = db['users'].find_one()

```json
{
  "_id": 111499, 
  
  "first_name": "Андрей", 
  "sex": 2, 
  "bdate": "23.9.1999", 
  
  "about": "Жизнелюб", 
  "interests": "Немного спорта", 
  "movies": "Те которые можно назвать хитами в кинематографе", 
  "tv": "НЕТ", 
  "books": "ФИЛОСОФИЯ И ПСИХОЛОГИЯ", 
  "games": "Логика", 
  "quotes": "… лучшая доля не воздерживаться от наслаждений,\n
             А в том, чтобы властвовать над ними,\n
             Не подчиняясь им.  АРИСТИПП\n", 
  
  "university_name": "ИФЭП ОЗ", 
  "education_status": "Выпускник (специалист)", 
  "universities": [
    {
      "city": 1, 
      "name": "ИФЭП ОЗ", 
      "country": 1, 
      "education_form": "Заочное отделение", 
      "graduation": 2006, 
      "education_status": "Выпускник (специалист)", 
      "faculty_name": "Юриспруденция", 
      "faculty": 120881, 
      "id": 65746
    }
  ], 
  
  "occupation": {
    "type": "university", 
    "id": 65746, 
    "name": "ИФЭП ОЗ"
  }, 

  "city": {
    "id": 1, 
    "title": "Москва"
  }, 
  
  "personal": {
    "smoking": 3, 
    "political": 8, 
    "alcohol": 4
  }, 

  ...
}
```

##### Как вывести JSON в читаемом формате?

In [8]:
import json

print json.dumps(user, indent=4, encoding='utf8', ensure_ascii=False)[:500] + '...'

{
    "interests": "Немного спорта", 
    "domain": "soop81", 
    "can_see_audio": 1, 
    "university_name": "ИФЭП ОЗ", 
    "site": "", 
    "education_status": "Выпускник (специалист)", 
    "quotes": "… лучшая доля не воздерживаться от наслаждений,\nА в том, чтобы властвовать над ними,\nНе подчиняясь им.                                                                                   АРИСТИПП\n", 
    "can_see_all_posts": 1, 
    "relation": 0, 
    "occupation": {
        "type": "univers...


## Итерация по объектам коллекции

Итерация по объектам коллекции делается при помощи **курсоров**.

Курсор (в терминах MongoDB) — объект, который содержит результат запроса к базе данных, т.е. последовательность найденных документов. Найденные документы загружаются из базы данных последовательно, по необходимости.

Запросы к коллекции MongoDB делаются при помощи метода `collection.find()`, который и возвращает курсор. По-умолчанию метод `find()` находит все имеющиеся документы в коллекции.

##### Создадим курсор, указывающий на все документы в коллекции

In [9]:
users_collection = db['users']

cursor = users_collection.find()

##### Количество документов, на которые указывает курсор

In [10]:
cursor.count()

730727

##### Найденный документ по его порядковому номеру (в списке найденных)

In [11]:
cursor[4]

{u'_id': 149589,
 u'about': u'',
 u'activities': u'123\u0444\u043a\u0432\u043f\u0440\u044b\u043a\u043f\u0440\u0432\u0430',
 u'bdate': u'1.1.1998',
 u'books': u'',
 u'can_post': 1,
 u'can_see_all_posts': 1,
 u'can_see_audio': 1,
 u'can_write_private_message': 1,
 u'city': {u'id': 1, u'title': u'\u041c\u043e\u0441\u043a\u0432\u0430'},
 u'common_count': 0,
 u'country': {u'id': 1, u'title': u'\u0420\u043e\u0441\u0441\u0438\u044f'},
 u'domain': u'id149589',
 u'faculty': 0,
 u'faculty_name': u'',
 u'first_name': u'\u0421\u0430\u043d\u044f',
 u'games': u'',
 u'graduation': 0,
 u'has_mobile': 0,
 u'interests': u'\u0432\u043e\u043f\u0440\u043e\u0430\u043b\u0430\u0430\u0440',
 u'last_seen': {u'platform': 7, u'time': 1303674215},
 u'movies': u'hkfhkfhfh',
 u'music': u'mvhjmfkfhkfhkf',
 u'occupation': {u'id': 106,
  u'name': u'\u0412\u0423 \u041c\u041e \u0420\u0424 (\u0431\u044b\u0432\u0448. \u0412\u041f\u0410 \u0438\u043c. \u041b\u0435\u043d\u0438\u043d\u0430)',
  u'type': u'university'},
 u'onli

##### Можно взять первые несколько документов из курсора

Для этого используется метод `.limit(n)`, который возвращает курсор, указывающий на первые n документов.

In [12]:
first_records = cursor.limit(10)

##### Загрузим все документы из курсора в python-список

Обратите внмание на то, что эту операцию можно делать только в случае, если вы уверены в том, что найденных документов мало и они помещаются в память

In [13]:
list_of_records = list(first_records)

##### Итерация по документам в цикле

In [14]:
def process(doc):
    pass
    
for doc in cursor:
    process(doc)

**Обратите внимание!** После того как курсор передал в цикле все найденные документы, он больше не будет итерироваться по ним, придется создавать новый курсор при помощи метода `find()`.

## Язык запросов MongoDB

Полезные ссылки для ознакомления:
  - [Вики-статья с описанием языка запросов](http://softtime.info/view/Язык_запросов_MongoDB)
  - [Статья на Хабре про запросы к MongoDB](https://habrahabr.ru/post/134590/)
  - [Примеры запросов (на английском)](https://docs.mongodb.org/manual/reference/sql-comparison/)
  - [Оригинальная документация по методу `find()` (на английском)](https://docs.mongodb.org/v3.0/reference/method/db.collection.find/)

##### Метод `find()`

Синтаксис:
```python
collection.find(filter, projection, limit=N, skip=N, sort=<sort order>)
```

### Фильтрация

##### Сколько пользователей с именем "Василий"?

In [15]:
users_collection.find({'first_name': u'Василий'}).count()

1146

##### Василиев, у которых указан университет?

In [16]:
cursor = users_collection.find({
        'first_name': u'Василий',
        'university_name': {'$exists': True},
    })
cursor.count()

933

##### Сколько пользователей с числом друзей, большим 9000?

In [17]:
friends_collection = db['friends']

In [18]:
friends_collection.find({'response.count': {'$gt': 9000}}).count()

36

##### Список таких пользователей

In [19]:
friends_9k_plus = list(friends_collection.find({'response.count': {'$gt': 9000}}))

### Проецирование

Проецирование означает что из найденных документов нужно взять только определенные поля (те, которые необходимы для обработки) — это позволяет очень сильно экономить память и выигрывать в скорости обработки данных. Всегда указывайте параметр `projection`, если вам не нужны все поля из документа.

##### К примеру, мы хотим вывести только названия университетов выбранных пользователей

In [21]:
query = {'university_name': {'$exists': True}}

for user in users_collection.find(query, {'university_name': 1}, limit=20):
    print 'User', user['_id'], ' university:', user['university_name']

User 111499  university: ИФЭП ОЗ
User 149589  university: ВУ МО РФ (бывш. ВПА им. Ленина)
User 259973  university: 
User 337203  university: АФСБ РФ
User 392936  university: АПИ при ИГиП РАН
User 421623  university: 
User 439255  university: 
User 514288  university: РГУП (бывш. РАП)
User 644223  university: 
User 704101  university: Финансовый университет (ФА)  при Правительстве РФ
User 838187  university: Финансовый университет (ФА)  при Правительстве РФ
User 859548  university: ДА МИД РФ
User 867882  university: МГТУ им. Н. Э. Баумана
User 889859  university: МГСУ НИУ (МГСУ-МИСИ)
User 793098  university: РУДН
User 922143  university: НИЯУ МИФИ
User 945084  university: 
User 954283  university: 
User 957547  university: МИР
User 975981  university: MHH


## Составление таблицы

Анализировать данные удобнее всего, представив их в табличном виде. Представить документы, полученные по запросу в MongoDB в виде таблицы можно при помощи метода `pandas.DataFrame.from_records()`:

In [29]:
query = {
        'university_name': u'МГУ',
    }

projection = {
    'university_name': 1,
    'first_name': 1,
    'bdate': 1,
    'city.title': 1,
}

cursor = users_collection.find(query, projection)

##### Ожидаемое число строчек в таблице

In [30]:
cursor.count()

71449

##### Составляем таблицу из найденных JSON-документов

In [31]:
import pandas
df_msu_students = pandas.DataFrame.from_records(cursor)

In [32]:
df_msu_students.head()

,_id,bdate,city,first_name,university_name
0,1184434,24.3.1999,{u'title': u'Москва'},Захар,МГУ
1,1912115,22.6.1998,{u'title': u'Москва'},Наталья,МГУ
2,1923901,1.2.1998,{u'title': u'Москва'},Дарья,МГУ
3,2187145,26.5.1998,{u'title': u'Москва'},Вадим,МГУ
4,3235498,21.4.2001,{u'title': u'Москва'},Роберт,МГУ


## Задание

При помощи языка запросов и разобранных методов работы с MongoDB определите, у скольких пользователей из всей коллекции доступна информация со стены?

Подсказки:
  - информация со стен находится в коллекции `walls`
  - если стена доступна, то документ, соответствующий пользователю содержит поле `response`, иначе — поле `response` отсутствует, а вместо него в поле `error` записана информация об ошибке загрузки информации со стены